# Tutorial 3: Memory Integration

Learn how Aragora's memory systems help improve debates over time.

**What you'll learn:**
- Using CritiqueStore to learn from past debates
- Multi-tier memory (Continuum) for context retention
- Pattern recognition and retrieval
- ELO rankings for agent skill tracking

**Time:** ~10 minutes

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from aragora import Arena, Environment, DebateProtocol
from aragora.agents.base import create_agent
from aragora.memory.store import CritiqueStore
from aragora.memory.continuum import ContinuumMemory
from aragora.ranking.elo import ELOTracker


## CritiqueStore: Learning from Past Debates

The CritiqueStore persists debate outcomes and extracts patterns that can improve future debates.

In [ ]:
# Create a critique store (uses SQLite by default)
store = CritiqueStore("tutorial_memory.db")

# Check statistics
stats = store.get_stats()

## Running Debates with Memory

When you pass a CritiqueStore to the Arena, it automatically stores results and learns patterns.

In [ ]:
# Create agents
agents = [
    create_agent("anthropic-api", "claude", "proposer"),
    create_agent("openai-api", "gpt", "critic")
]

# Create environment
env = Environment(
    task="What's the best approach for database schema versioning?"
)

# Create protocol
protocol = DebateProtocol(rounds=2, consensus="majority")

# Run debate WITH memory
arena = Arena(env, agents, protocol, memory=store)
result = await arena.run()

# Store the result
store.store_debate(result)


## Retrieving Learned Patterns

Patterns are extracted from debates and can be used to inform future discussions.

In [ ]:
# Retrieve patterns
patterns = store.retrieve_patterns(
    issue_type="architecture",  # Filter by type
    min_success=1,               # Minimum times pattern was successful
    limit=5                      # Max patterns to return
)

for p in patterns:
    if p.suggestion_text:
        pass

## Continuum Memory: Multi-Tier Context

ContinuumMemory provides multiple tiers for different retention periods:

| Tier | TTL | Purpose |
|------|-----|--------|
| Fast | 1 min | Immediate context |
| Medium | 1 hour | Session memory |
| Slow | 1 day | Cross-session learning |
| Glacial | 1 week | Long-term patterns |

In [ ]:
# Create multi-tier memory
continuum = ContinuumMemory()

# Store data in different tiers
continuum.store(
    key="current_task",
    value={"task": "Design a rate limiter", "started": "2024-01-01"},
    tier="fast"  # Will expire in 1 minute
)

continuum.store(
    key="session_context",
    value={"project": "E-commerce API", "team": "Backend"},
    tier="medium"  # Will expire in 1 hour
)

continuum.store(
    key="learned_pattern",
    value={"pattern": "Token bucket is preferred for rate limiting"},
    tier="slow"  # Will expire in 1 day
)


In [ ]:
# Retrieve data (searches all tiers by default)
current = continuum.retrieve("current_task")

# Retrieve from specific tier
pattern = continuum.retrieve("learned_pattern", tier="slow")

# Search across tiers
results = continuum.search("rate", limit=5)

## Getting Context for Debates

Use `get_context()` to retrieve relevant memory for a new task.

In [ ]:
# Get context relevant to a new task
context = continuum.get_context(
    task="Design an API rate limiter for high traffic",
    limit=5
)


## ELO Rankings: Agent Skill Tracking

ELO rankings track agent performance over time, helping select the best agents for tasks.

In [ ]:
# Create ELO tracker
elo = ELOTracker(db_path="tutorial_elo.db")

# Get current rankings
rankings = elo.get_rankings()

for agent, rating in rankings[:10]:
    pass

In [ ]:
# Record a debate outcome
elo.record_debate(
    winner="claude-proposer",
    loser="gpt-critic",
    draw=False  # Set to True for ties
)

# Get updated rankings
for agent, rating in elo.get_rankings()[:5]:
    pass

## Putting It All Together

Run a debate with full memory integration.

In [ ]:
# Get context from previous debates
historical_context = store.get_relevant_context(
    task="Design an authentication system",
    limit=3
)

# Create environment with context
env = Environment(
    task="Design an authentication system for a mobile banking app",
    context=f"Previous learnings: {historical_context}"
)

# Create arena with memory
arena = Arena(env, agents, protocol, memory=store)

# Run debate
result = await arena.run()

# Store result and update ELO
store.store_debate(result)
if result.consensus_reached:
    elo.record_debate(
        winner=result.winning_agent,
        loser=result.dissenting_agents[0] if result.dissenting_agents else None
    )


## Next Steps

- **Tutorial 4**: Deploy Aragora in production
- **Tutorial 5**: Advanced features (Gauntlet, WebSocket streaming)